In the last file, we used dataframe manipulation tools to remove data that we won't need for our analysis, to create new variables, and to change data types.

In this file, we'll learn how to combine dataframes into one single, clean dataframe for further analysis.

We saved the data we made changes to in the last file in `.csv` files:

* `sat_results_2.csv`
* `ap_2010_2.csv`
* `class_size_2.csv`
* `demographics_2.csv`
* `graduation_2.csv`
* `hs_directory_2.csv`

**Task**

* Let's import them into R and get started with the next steps in our data cleaning process.

**Answer**

`library(readr)
sat_results <- read_csv("sat_results_2.csv")
ap_2010 <- read_csv("ap_2010_2.csv") 
class_size <- read_csv("class_size_2.csv")
demographics <- read_csv("demographics_2.csv") 
graduation <- read_csv("graduation_2.csv") 
hs_directory <- read_csv("hs_directory_2.csv")`

So far, we've performed a variety of data cleaning operations on the dataframes:

* Simplified the dataframes to contain only variables and observations that we need for our analysis.
* Created new variables from existing ones using calculations and string manipulation.

Now that the six individual dataframes have been cleaned, the next step is to combine them.

We learned to combine vectors and matrices by rows and columns using `rbind()` and `cbind()`.

When we combined vectors and matrices, we bond values of rows or columns together by position. We learned that if vectors are of different lengths, the recycling rule dictates that the shorter vector is repeated.

We can combine dataframes by using the same ways with the [functions `bind_rows()` and `bind_cols()`](https://dplyr.tidyverse.org/reference/bind.html), which work similarly to `rbind()` and `cbind()`.

In contrast, when we combine a pair of dataframes, we can bind columns together based on a variable's matching values rather than on observation positions. The variable used to connect each pair of tables is called a **key**. In this case, we will combine dataframes based on the key variable `DBN`.

To combine the dataframes, we'll use `dplyr` tools for working with multiple tables of data. When we have multiple tables containing data that we are interested in relationships between, they are collectively referred to as [relational data](https://en.wikipedia.org/wiki/Relational_database).

The six dataframes we have been working with are considered relational data. This because we are interested in **relationships** between variables in different tables, such as SAT score and average class size.

We'll use the `dplyr` functions to combine the tables for performing [mutating joins](https://dplyr.tidyverse.org/reference/join.html), which add new variables to one dataframe based on matching observations in another dataframe.

If we are familiar with how joining data tables works in [SQL](https://en.wikipedia.org/wiki/SQL), the `dplyr` functions for joining tables will seem very familiar.

Before joining dataframe, for our analysis, we have to ensure that each dataframe has one observation - that is, one row - for each school. The `DBN` variable must be unique to each school. To successfully combine the dataframes, we should have no more than one instance of each `DBN` value per dataframe.

There are various ways we could check for duplicate values of DBN within each dataframe. However, we'll learn one that we find to be incredibly efficient, using the `duplicated()` function.

`duplicated()` is a base R function, so we don't need to load any new packages to use it. The function takes a vector as input and returns a logical vector that indicates whether the value is a duplicate of one that comes before it in the vector.

We could apply the `duplicated()` function to each dataframe's `DBN` column, for example:

`duplicated(sat_results$DBN)`

We could use the `sum()` function to check how many duplicated values we have:

`sum(duplicated(vector))`


**Task**

* Let's use the same technique to count the number of duplicated values for our datasets.

**Answer**

`sat_results_duplicated <- sum(duplicated(sat_results$DBN))
ap_2010_duplicated <- sum(duplicated(ap_2010$DBN))
class_size_duplicated <- sum(duplicated(class_size$DBN))
demographics_duplicated <- sum(duplicated(demographics$DBN))
graduation_duplicated <- sum(duplicated(graduation$DBN)) 
hs_directory_duplicated <- sum(duplicated(hs_directory$DBN))` 

`duplicate_DBN <- list( "sat_results" = sat_results_duplicated, 
                      "ap_2010" = ap_2010_duplicated, 
                      "class_size" = class_size_duplicated,
                      "demographics" = demographics_duplicated, 
                      "graduation" = graduation_duplicated, 
                      "hs_directory" = hs_directory_duplicated)`

We have now generated a list of rows from each dataframe that contain duplicate values of `DBN`.

Only `ap_2010` has a duplicate DBN value.

To remove the duplicated rows from `ap_2010` we can use the `dplyr` [function `distinct`()](https://dplyr.tidyverse.org/reference/distinct.html). Its syntax is as follows:

`dataframe %>% distinct(the_name_of_the_column, .keep_all = TRUE)`
    
The `.keep_all = TRUE` parameter allows keeping all the columns in the dataframe.

**Task**

* Let's use this technique to remove the `DBN` duplicated row from `ap_2010`.

**Answer**

`ap_2010 <- ap_2010 %>% distinct(DBN, .keep_all = TRUE)`

As we combine the six NYC schools dataframes, we will begin with one dataframe and then add variables from the other dataframes to it by matching them up with the values of **DBN**, the key. Recall that the term for joins that add new variables to a dataframe based on matching observations in another dataframe is mutating joins. There are two main types of **mutating joins** that we will discuss:

* Inner Joins
* Outer Joins

Let's start with `inner joins`, which match pairs of variables in two dataframes whenever their values of the key are the same. The resulting dataframe does **not** include any rows with unmatched keys:

Inner joins are useful when we only want to **include results that appear in both of the tables** we're joining. For example, suppose we're interested in using our data to make a scatter plot to understand the relationship between class size and how well students score on the SAT. In that case, our new dataframe probably doesn't need to include data on total SAT scores that doesn't have a matching class size value.

To combine two tables using an inner join, we can use the `dplyr` [function `inner_join()`](https://dplyr.tidyverse.org/reference/join.html). The syntax involves beginning with one dataframe and specifying the dataframe we want to join with it. To specify the identity of the key we'll use for the join, use `by = "DBN"`:

`sat_results %>% inner_join(class_size, by = "DBN")`

**Task**

* Use the `inner_join()` function to combine the `sat_results` and `class_size` dataframes.

* Create a scatter plot with `avg_class_size` on the x-axis and `avg_sat_score` on the y-axis to see if SAT scores seem to be related to class size.


**Answer**

`sat_class_size <- sat_results %>%  inner_join(class_size, by = "DBN")`

`ggplot(data = sat_class_size) +
      aes(x = avg_class_size, y = avg_sat_score) +
      geom_point()`

Now that we've learned about using inner joins to combine dataframes, let's move on to discussing the other type of mutating join we'll be working with: Outer joins.

**Outer joins** keep observations that appear in at least one of the two tables we're combining. Outer joins can be divided into **three types**:

* Left joins
* Right joins
* Full joins

To illustrate how these different types of outer joins work, we'll once again use abbreviated versions of the `sat_results` and `class_size` dataframes as examples.

Performing a left join keeps all observations in the dataframe on the left (`sat_results`) and drops observations from the dataframe on the right (`class_size`) that have no key (`DBN`) match.

Performing a **right join** keeps **all observations in the dataframe on the right** (`class_size`) and drops observations from the dataframe on the left (`sat_results`) that have no key (`DBN`) match

Performing a **full join** keeps **all observations from both** the `sat_results` and `class_size` dataframes, and fills in missing variables with "NA"

The syntax for combining dataframes using outer joins is similar to that which we used to perform inner joins. The `dplyr` functions to perform the different types of outer joins are:

* `left_join()`
* `right_join()`
* `full_join()`

As with the `inner_join()` function, when using the three functions for performing outer joins, use `by = "DBN"` to specify the key's identity.

`sat_results %>% left_join(class_size, by = "DBN")`

These three types of outer joins are useful under different circumstances, and which one we choose will depend on our data analysis needs.

For example, let's say we mainly are interested in how class size (`avg_class_size` in the `class_size` dataframe) and the percentage of a school's students learning English (`ell_percent` in the demographics dataframe) are related to scores on the writing portion of the SAT. When we combine the `sat_score`, `class_size`, and `demographics` dataframes, we may choose to join `class_size` and `demographics` to `sat_score` using `left_join()`, so we retain all class size and demographics data that match an SAT score.

We can use the `%>%` piping operator to chain multiple types of join functions together. To join `class_size` and `demographics` to `sat_results` as described above, we could write:

`combined <- sat_results %>%
  left_join(class_size, by = "DBN") %>%
  left_join(demographics, by = "DBN")`

It's now time to combine the six dataframes to create a single, clean one to use for analyses.

As we decide which types of joins to use to combine the dataframes, let's briefly revisit our goal for this analysis: We will use the data to understand how NYC high schools' demographics (race, sex, income, etc.) affect how well students perform academically.

We'll start with the data that provides us with information about students' academic performance: `sat_results` and `ap_2010`. Some high schools may have data only for either SAT scores or AP exams. To make sure we retain all observations from these two dataframes, let's combine `ap_2010` with `sat_results` using a **full join**.

Next, let's consider which type of join we should use to add the `class_size`, `demographics`, `graduation`, and `hs_directory` data to the new dataframe we're building. We want to retain all observations in `sat_results` and `ap_2010`. Thus, it makes sense to use left joins, so only values of the `class_size`, `demographics`, `graduation`, and `hs_directory` dataframes will be included in the new dataframe.

**Task**

* Use `full_join()` to join `ap_2010` to `sat_results`.

* Use `left_join()` to add `class_size`, `demographics`, `graduation`, and `hs_directory` to the new dataframe.

**Answer**

`combined <- sat_results %>%
  full_join(ap_2010, by = "DBN") %>%
  left_join(class_size, by = "DBN") %>%
  left_join(demographics, by = "DBN") %>%
  left_join(graduation, by = "DBN") %>%
  left_join(hs_directory, by = "DBN")`

In this file, we worked with functions for different types of mutating joins (inner and outer joins). We used them to combine the six dataframes that we've been working with into a single, clean one for future analysis.

In the next file, we'll begin working with the cleaned dataframe we created to ask and answer questions about how demographic factors affect NYC schools' academic performance. We'll use exploratory data visualization techniques, and we'll learn to quantify the strength of relationships between variables using **correlation**. As we begin our analyses, we'll also learn about a useful concept for data cleaning and analysis: **Reshaping data**.